In [ ]:
import requests
root = 'https://deslabs.ncsa.illinois.edu/easyweb/api'
import pandas as pd

### Create a new token

In [ ]:
post_data = {
    'username': 'your-dessci-username',
    'password': 'your-dessci-password'
}

In [ ]:
req = requests.post(root+'/v1/token/',data=post_data, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))
token = req.json()['token']

### Check Token status

In [ ]:
req = requests.get(root+'/v1/token/?token={}'.format(token), verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))

### Query Job

In [ ]:
#query
q = "select * from (select ra,dec from y3a2_coadd_object_summary) where rownum < 10"
# create body of POST request
body = {
    'token'       : token,
    'query'       :  q,
    'db'          : 'dessci',
    'output'      : 'test.csv',
    'compression' : 'no',
    'email'       : '',
    'jobname'     : 'api_test2'
}

In [ ]:
req = requests.post(root+'/v1/query/', data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))
job = req.json()['jobid']

### Check Job by its id

In [ ]:
body = {
    'token'       : token,
    'jobid'       : job
}

In [ ]:
req = requests.post(root+'/v1/jobs/', data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))

In [ ]:
for f  in req.json()['files']:
    print(f)

### Check all Jobs

In [ ]:
body = {
    'token'       : token,
    'jobid'       : 'all'
}

In [ ]:
req = requests.post(root+'/v1/jobs/', data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))

In [ ]:
df = pd.DataFrame.from_dict(req.json())
df = df.drop(['msg', 'status'], axis=1)

In [ ]:
df

### Delete a Job

In [ ]:
body = {
    'token'       : token,
    'jobid'       : job
}

In [ ]:
req = requests.delete(root+'/v1/jobs/', data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))

### Delete All Jobs

In [ ]:
body['jobid'] = 'all'
all_jobs = requests.post(root+'/v1/jobs/', data=body, verify=False)

In [ ]:
for j in range(len(all_jobs.json()['job_id'])):
    body['jobid'] = all_jobs.json()['job_id'][j]
    req = requests.delete(root+'/v1/jobs/', data=body, verify=False)
    print(req.text)

## Cutouts (with ra/dec input list)

In [ ]:
ra = [36.5278, 30.8839]
dec = [-10.0994, -38.0092]
xs = 1.0
ys = 1.0
# create body of request
body = {
    'token'        :  token,        # required
    'ra'           : str(ra),       # required of coadd/coord/csvfile
    'dec'          : str(dec),      # required of coadd/coord/csvfile
    'xsize'        : str(xs),       # required (default : 1.0)
    'ysize'        : str(ys),       # required (default : 1.0)
    'jobname'      : 'test ra/dec list',        # optional (default : 'false') 'true': will not generate pngs (faster)
    'make_tiffs'   : 'false',       # optional
    'make_pngs'    : 'true',        # optional
    'make_fits'    : 'false',       # optional
    'colors'       : 'g,r,i,z,y',   # required if make_fits is true; pick any or all bands
}
req = requests.post(root+'/v1/cutout/',data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))
job = req.json()['jobid']

## Cutouts (with coadd input list)

In [ ]:
coadd = [353115749, 353099273]
xs = 1.0
ys = 1.0
# create body of request
body = {
    'token'        :  token,        # required
    'coadd'        : str(coadd),    # required of coadd/coord/csvfile
    'xsize'        : str(xs),       # required (default : 1.0)
    'ysize'        : str(ys),       # required (default : 1.0)
    'jobname'      : 'test coadd list',        # optional (default : 'false') 'true': will not generate pngs (faster)
    'make_tiffs'   : 'false',       # optional
    'make_pngs'    : 'true',        # optional
    'make_fits'    : 'false',       # optional
    'colors'       : 'g,r,i,z,y',   # required if make_fits is true; pick any or all bands
}
req = requests.post(root+'/v1/cutout/',data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))
job = req.json()['jobid']

## Cutouts (with CSV input)

In [ ]:
csvfile = '/path/to/csv/file.csv'
files = {'csvfile' : ('cf', open(csvfile, 'rb'), 'text/csv', {'Expires':'0'})}
xs = 1.0
ys = 1.0
# create body of request
body = {
    'token'        :  token,        # required
    'csvfile'      :  csvfile,      # required of coadd/coord/csvfile
    'xsize'        : str(xs),       # required (default : 1.0)
    'ysize'        : str(ys),       # required (default : 1.0)
    'jobname'      : 'test csvfile',        # optional (default : 'false') 'true': will not generate pngs (faster)
    'make_tiffs'   : 'false',       # optional
    'make_pngs'    : 'true',        # optional
    'make_fits'    : 'false',       # optional
    'colors'       : 'g,r,i,z,y',   # required if make_fits is true; pick any or all bands
}
req = requests.post(root+'/v1/cutout/',data=body, files=files, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))
job = req.json()['jobid']

## Finding Chart (with ra/dec input list)

In [ ]:
ra = [36.5278, 30.8839]
dec = [-10.0994, -38.0092]
xs = 1.0
ys = 1.0
magnitude_limit = 20.0
# create body of request
body = {
    'token'        :  token,        # required
    'ra'           : str(ra),       # required of coadd/coord/csvfile
    'dec'          : str(dec),      # required of coadd/coord/csvfile
    'xsize'        : str(xs),       # required (default : 1.0)
    'ysize'        : str(ys),       # required (default : 1.0)
    'mag_limit'    : str(magnitude_limit),      # required; recommended 20.0
    'jobname'      : 'test ra/dec list',        # optional (default : 'false') 'true': will not generate pngs (faster)
    'colors'       : 'i',           # required; pick any or all bands
    'return_cut'   : 'true',        # required
}
req = requests.post(root+'/v1/chart/',data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))
job = req.json()['jobid']

## Finding Chart (with CSV input)

In [ ]:
csvfile = '/path/to/csv/file.csv'
files = {'csvfile' : ('cf', open(csvfile, 'rb'), 'text/csv', {'Expires':'0'})}
xs = 1.0
ys = 1.0
magnitude_limit = 20.0
# create body of request
body = {
    'token'        :  token,        # required
    'csvfile'      :  csvfile,      # required of coadd/coord/csvfile
    'xsize'        : str(xs),       # required (default : 1.0)
    'ysize'        : str(ys),       # required (default : 1.0)
    'mag_limit'    : str(magnitude_limit),      # required; recommended 20.0
    'jobname'      : 'test csvfile',            # optional (default : 'false') 'true': will not generate pngs (faster)
    'colors'       : 'i',           # required; pick any or all bands
    'return_cut'   : 'true',        # required
}
req = requests.post(root+'/v1/chart/',data=body, files=files, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))
job = req.json()['jobid']

In [ ]:
body = {
    'token'       : token,
    'jobid'       : job
}

In [ ]:
req = requests.post(root+'/v1/jobs/', data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))

In [ ]:
for f  in req.json()['files']:
    print(f)